# Endian Classifier

## Prerequisites

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers[torch] -U
!pip install datasets -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import torch

# Get device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

Device: cuda


## Load Dataset

In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd

# Load dataset
dataset = load_dataset("ryfye181/endianness", data_dir="test", split='train')

In [ ]:
small_ds = dataset.take(1000)

In [ ]:
small_ds

Dataset({
    features: ['data', 'endianness'],
    num_rows: 1000
})

## Load Model and Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "ryfye181/distilbert_endian_classifier"

In [ ]:
# Tokenize dataset

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_dataset(dataset):
    return tokenizer(dataset["data"], padding="max_length", truncation=True)
ds_test_tokenized = small_ds.map(tokenize_dataset, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

## Load pipeline for sequence classification

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device)

In [ ]:
index = 902
test_data, test_label = small_ds[index]["data"], small_ds[index]["endianness"]
print(f"Data column = {test_data}")
print(f"Label = {test_label}")

Data column = 00 526b 5449 0000 0000 556b 0050 726b 0069 6974 0063 2d5b 2065 616e 656d 5d73 5b20 6f2d 6420 7269 205d  2d5b 2052 616e 656d 205d 2d5b 5b76 5d6e 205d 2d5b 5d56 5b20 772d 6e5b 5d5d 5b20 312d 4361 6344 4766 4967 4c4b 724e 5473 5574 5d78 7320 756f 6372 2d65 6966 656c  000a 0000 704f 6974 6e6f 3a73 200a 2d20 2030 2020 2020 2020 2020 6f66 6d72 7461 7420 6172 736e 616c 6974 6e6f 6f20 7475 7570 2074 6c61 206c 6163 6170 6962 696c  6974 7365 6f20 206e 6e6f 2065 696c 656e 200a 2d20 2031 2020 2020 2020 2020 6f66 6d72 7
Label = big


In [ ]:
test_data = small_ds[index]["data"]
classifier(test_data)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[{'label': 'little', 'score': 0.5135465860366821}]